In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

data_anly= pd.read_csv('data/data_cal.csv')
data_anly

,Datetime,Measured & Upscaled,Most recent forecast,Monitored capacity,ShiftMeasured,Fluctuation15M,PredFluctuation15M,ShiftFluctuation15M
0,2022-01-01 00:00:00,1803.48,1881.52,2254.4,1803.48,0.00,78.04,0.00
1,2022-01-01 00:15:00,1962.39,1855.50,2254.4,1803.48,158.91,52.02,0.00
2,2022-01-01 00:30:00,1801.63,1823.16,2254.4,1962.39,-160.76,-139.23,158.91
3,2022-01-01 00:45:00,1824.41,1795.96,2254.4,1801.63,22.78,-5.67,-160.76
4,2022-01-01 01:00:00,1661.57,1779.28,2254.4,1824.41,-162.84,-45.13,22.78
...,...,...,...,...,...,...,...,...
35035,2022-12-31 22:45:00,2187.05,2133.89,2254.4,2177.33,9.72,-43.44,16.48
35036,2022-12-31 23:00:00,2177.68,2123.49,2254.4,2187.05,-9.37,-63.56,9.72
35037,2022-12-31 23:15:00,2180.85,2129.58,2254.4,2177.68,3.17,-48.10,-9.37
35038,2022-12-31 23:30:00,2177.87,2147.49,2254.4,2180.85,-2.98,-33.36,3.17


In [3]:
param = {'ratio':0.1,
         'C_C':0.3*2254.4,
         'C_W':2254.4, # MW
         'SOE_init':0.5*225.44,
         'SOE_max':0.95*225.44,
         'SOE_min':0.05*225.44,
         'P_BSS_max':225.44,
         'P_BSS_min':-225.44,
         'PT10M':1/4,
         'refine_power':50,
         'slight_upward':6,# 向上波动四分位数
         'slight_downward':-7, # 向下波动四分位数
         'condition1':"upward_fluctuation",
         'condition2':"downward_fluctuation",
         'condition3':"upward_slight_fluctuation",
         'condition4':"downward_slight_fluctuation",
         'condition5':"no_fluctuation",
         'nita_c': 0.932,
         'nita_d': 0.932}
        #  'P_d_init':0,
        #  'P_c_init':0,


- 编程日志
- 由于充放电效率的区别必须放置01变量来区分充放电
- 目前的三步并不能满足想象中的调节效果
- 在这里区分一下变量的定义.
- P_c 第k步的充电功率
- P_d 第k步的放电功率
- E 执行完第k步放电功率的能量状态
- v_c 第k步的充电标签
- v_d 第k步的放电标签

- 22.30已实现标准的方法
- 22.31开始尝试滚动优化
- 0404 1440 滚动已经实现，现在尝试消去01变量
- 0404 1625 滚动已经正式实现
- 0404 2200 实际数据以及实际参数以及配置进去 实现了两个方案 而且窗口越长显然波动降低越好

In [4]:
# 导入docplex
from docplex.mp.model import Model
# 创建模型对象

def solver(P_W_list,n,M,mode,power,cap):
    # 函数功能：求解最佳储能功率
    # 函数输入: P_W_list风机功率列表，n为总步数，M为窗口宽度
    # 函数输出：P_BSS_rolling储能滚动优化出力功率列表,E_rolling储能滚动优化能量状态列表
    
    P_BSS_rolling = []
    E_rolling = []
    real_steps = n - M
    P_optimize = P_W_list
    # 添加约束
    for real_step in tqdm(range(0, real_steps), desc='Optimizing'):
        # print(f"第{real_step+1}次滚动求解")
        sub_list = P_W_list[real_step:real_step+M]

        # 新建求解模型
        cplex_obj = Model()

        # 限制变量范围
        P_BSS = [cplex_obj.continuous_var(name='P_BSS[%d]' % i, lb=-power, ub=power) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1
        z_c = [cplex_obj.binary_var(name='z_c[%d]' % i) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1
        z_d = [cplex_obj.binary_var(name='z_d[%d]' % i) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1
        P_BSS_plus = [cplex_obj.continuous_var(name='P_BSS_plus[%d]' % i, lb=0, ub=power) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1
        P_BSS_minus = [cplex_obj.continuous_var(name='P_BSS_minus[%d]' % i, lb=0, ub=power) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1

        # 创建由于从0到n-1有n个能量状态，所以有n-1条状态传递方程，还有对0状态的等式约束
        
        E = [cplex_obj.continuous_var(name='E[%d]' % i, lb=0.05*cap, ub=0.95*cap) for i in range(M)]
        
        # 确保 P_BSS_plus 和 P_BSS_minus 正确表示 P_BSS 的正负部分
        for i in range(M): # param['P_BSS_max']
            cplex_obj.add_constraint(P_BSS[i] == P_BSS_plus[i] - P_BSS_minus[i])
            cplex_obj.add_constraint(P_BSS_plus[i] <= power * z_c[i])  # 当 z_c[i]=0 时，P_BSS_plus 必须为 0
            cplex_obj.add_constraint(P_BSS_minus[i] <= power * z_d[i])  # 当 z_d[i]=0 时，P_BSS_minus 必须为 0
            cplex_obj.add_constraint(z_c[i] + z_d[i] <= 1)  # 确保不同时为正负
            
        # 添加初始状态，以等式约束形式
        if real_step == 0:
            cplex_obj.add_constraint(E[0] == 0.5*cap) # ,param['SOE_init']
            cplex_obj.add_constraint(P_BSS_plus[0] == 0) 
            cplex_obj.add_constraint(P_BSS_minus[0] == 0) 
            last_power = P_W_list[0] 

        else:
            cplex_obj.add_constraint(E[0] == last_state[1] - P_BSS_plus[0]/0.932*(1/4) + P_BSS_minus[0]*0.932*(1/4))

        # 等式约束
        for l in range(0,M-1):
            cplex_obj.add_constraint(E[l+1] == E[l] - P_BSS_plus[l+1]/0.932*(1/4) + P_BSS_minus[l+1]*0.932*(1/4)) # 电能量+是充电-是放电

        # 计算波动
        fluctuation = [((sub_list[i] + P_BSS_plus[i] - P_BSS_minus[i] ) - 
                (last_power if i == 0 else sub_list[i-1] + P_BSS_plus[i-1]- P_BSS_minus[i-1]))
               for i in range(0, M)]
        E_fluctuation = [(E[i]-E[i-1])for i in range(0, M)]
        E_half_fluctuation = [(E[i]-0.5*cap)for i in range(0, M)]
        # 计算波动的绝对值，并将其相加，作为目标函数
        if mode == 'square':
            objective_expr = cplex_obj.sum([fluctuation[i] * fluctuation[i]  for i in range(len(fluctuation))])
        elif mode == 'abs':
            objective_expr = cplex_obj.sum([cplex_obj.abs(fluctuation[i])  for i in range(len(fluctuation))])
        elif mode == 'range':
            objective_expr = cplex_obj.sum([cplex_obj.max( cplex_obj.abs(fluctuation[i])- 50,0)  for i in range(len(fluctuation))])
        elif mode == 'abs_with_E':
            objective_expr = cplex_obj.sum([0.7*cplex_obj.abs(fluctuation[i])+0.3*cplex_obj.abs(E_fluctuation[i])  for i in range(len(fluctuation))])
            # objective_expr = cplex_obj.sum([cplex_obj.max( cplex_obj.abs(fluctuation[i])- 50,0)  for i in range(len(fluctuation))])
        elif mode == 'abs_with_E0.5':
            objective_expr = cplex_obj.sum([0.7*cplex_obj.abs(fluctuation[i])+0.3*cplex_obj.abs(E_half_fluctuation[i])  for i in range(len(fluctuation))])
        elif mode == 'range_with_E0.5':
            objective_expr = cplex_obj.sum([cplex_obj.max( cplex_obj.abs(fluctuation[i])- 50,0) +0.3*cplex_obj.abs(E_half_fluctuation[i]) for i in range(len(fluctuation))])

        # 最大化目标函数
        cplex_obj.minimize(objective_expr)

        # cplex_obj.print_information()

        # 求解优化问题
        solution = cplex_obj.solve()
        # 获取结果
        if solution:
            # print(f"最优值为：{cplex_obj.objective_value:.2f}")
            p_bss_values = [round(P_BSS[i].solution_value,2) for i in range(M)]
            
            E_values = [round(E[i].solution_value,2) for i in range(M)]
            last_power = P_W_list[real_step] + P_BSS[0].solution_value
            last_state = [p_bss_values[0],E_values[0],last_power]

            P_optimize[real_step] = round(last_power,2)

            P_BSS_rolling.append(round(p_bss_values[0],2))
            E_rolling.append(round(E_values[0],2))
            # print(f'子序列为：{([P_optimize[0]] if real_step==0 else [P_optimize[real_step-1]])   +sub_list}')
            # print(f"P_BSS的取值为：{p_bss_values}")
            # print(f"E的取值为：{E_values}")
            # print()
        else:
            print("求解失败")
    
    return P_BSS_rolling,E_rolling



In [5]:
def generate_analyse_dataframe(datetime_list,P_list,p_bss_values,E_values):
    # 函数功能：将数据列表转化为分析的数据表
    # 函数输入：时间数据列（优化长度）、原功率数据列（优化长度）、储能功率数据列（优化长度）、能量数据列（优化长度）
    # 列表转换为DataFrame
    df_test = pd.DataFrame({
        'Datetime':datetime_list,
        'P_W_list': P_list,
        'p_bss_values': p_bss_values,
        'E':E_values
    })
    df_test['Datetime'] = pd.to_datetime(df_test['Datetime'])

    df_test['P_W_shift'] = df_test['P_W_list'].shift(1)
    df_test['P_W_shift'] = df_test['P_W_shift'].fillna(df_test['P_W_list'].iloc[0])# df_test['P_W_shift'].iloc[0]
    df_test['original_flucuation'] = df_test['P_W_list'] - df_test['P_W_shift']

    df_test['P_CW'] = df_test['P_W_list'] + df_test['p_bss_values']
    df_test['P_CW_shift'] = df_test['P_CW'].shift(1)
    df_test['P_CW_shift'] = df_test['P_CW_shift'].fillna(df_test['P_CW'].iloc[0])
    df_test['after_flucuation'] = df_test['P_CW'] - df_test['P_CW_shift']

    original_total_fluctuation = df_test['original_flucuation'].abs().sum()
    after_total_fluctuation = df_test['after_flucuation'].abs().sum()

    print(f'原波动是{original_total_fluctuation}')
    print(f'优化后的波动是{after_total_fluctuation}')

    return df_test



# 代价函数1

In [5]:
# 假设n是维度大小
# n = 100
n = len(data_anly)
M = 30
power = 0.3*2254.4
cap = 0.3*2254.4

P_W_list = 	list(data_anly['Measured & Upscaled'][:n]) # 对原数据进行截取
result_file_route = 'ch5_data/analyse_all_dates/abs.csv'

# P_W_list = [1000,1000,4000,1000,2500,5000,6000,4500,-50,10000,13000,500]
p_bss_values,E_values = solver(P_W_list,n,M,'abs',power,cap) 

# P_WC_values = list(np.array(p_bss_values) + np.array(P_W_list[:n-M]))
datetime_list = list(data_anly['Datetime'][:n-M]) 
P_W_list_original = list(data_anly['Measured & Upscaled'][:n-M])
df_test = generate_analyse_dataframe(datetime_list,P_W_list_original,p_bss_values,E_values)
df_test.to_csv(result_file_route)
df_test

Optimizing:   0%|          | 0/35010 [00:00<?, ?it/s]

Optimizing: 100%|██████████| 35010/35010 [40:19<00:00, 14.47it/s]   


原波动是1431452.8
优化后的波动是394225.9500000009


,Datetime,P_W_list,p_bss_values,E,P_W_shift,original_flucuation,P_CW,P_CW_shift,after_flucuation
0,2022-01-01 00:00:00,1803.48,0.00,338.16,1803.48,0.00,1803.48,1803.48,0.00
1,2022-01-01 00:15:00,1962.39,-466.09,446.76,1803.48,158.91,1496.30,1803.48,-307.18
2,2022-01-01 00:30:00,1801.63,-305.33,517.90,1962.39,-160.76,1496.30,1496.30,0.00
3,2022-01-01 00:45:00,1824.41,-328.11,594.35,1801.63,22.78,1496.30,1496.30,0.00
4,2022-01-01 01:00:00,1661.57,-165.26,632.86,1824.41,-162.84,1496.31,1496.30,0.01
...,...,...,...,...,...,...,...,...,...
35005,2022-12-31 15:15:00,2216.53,-68.80,505.66,2217.75,-1.22,2147.73,2146.72,1.01
35006,2022-12-31 15:30:00,2213.83,-64.81,520.76,2216.53,-2.70,2149.02,2147.73,1.29
35007,2022-12-31 15:45:00,2210.98,-61.02,534.98,2213.83,-2.85,2149.96,2149.02,0.94
35008,2022-12-31 16:00:00,2162.67,-11.70,537.71,2210.98,-48.31,2150.97,2149.96,1.01


# 代价函数2

In [6]:
# 假设n是维度大小
# n = 100
n = len(data_anly)
M = 1
power = 0.3*2254.4
cap = 0.3*2254.4

P_W_list = 	list(data_anly['Measured & Upscaled'][:n]) # 对原数据进行截取
result_file_route = 'ch5_data/analyse_all_dates/range_width_1.csv'

# P_W_list = [1000,1000,4000,1000,2500,5000,6000,4500,-50,10000,13000,500]
p_bss_values,E_values = solver(P_W_list,n,M,'range',power,cap) 

# P_WC_values = list(np.array(p_bss_values) + np.array(P_W_list[:n-M]))
datetime_list = list(data_anly['Datetime'][:n-M]) 
P_W_list_original = list(data_anly['Measured & Upscaled'][:n-M])
df_test = generate_analyse_dataframe(datetime_list,P_W_list_original,p_bss_values,E_values)
df_test.to_csv(result_file_route)
df_test

Optimizing: 100%|██████████| 35039/35039 [04:28<00:00, 130.31it/s]


原波动是1431730.85
优化后的波动是1031343.6300000006


,Datetime,P_W_list,p_bss_values,E,P_W_shift,original_flucuation,P_CW,P_CW_shift,after_flucuation
0,2022-01-01 00:00:00,1803.48,0.00,338.16,1803.48,0.00,1803.48,1803.48,0.0
1,2022-01-01 00:15:00,1962.39,-108.91,363.54,1803.48,158.91,1853.48,1803.48,50.0
2,2022-01-01 00:30:00,1801.63,51.85,349.63,1962.39,-160.76,1853.48,1853.48,0.0
3,2022-01-01 00:45:00,1824.41,29.07,341.83,1801.63,22.78,1853.48,1853.48,0.0
4,2022-01-01 01:00:00,1661.57,191.91,290.35,1824.41,-162.84,1853.48,1853.48,0.0
...,...,...,...,...,...,...,...,...,...
35034,2022-12-31 22:30:00,2177.33,40.42,122.00,2160.85,16.48,2217.75,2217.75,0.0
35035,2022-12-31 22:45:00,2187.05,30.70,113.77,2177.33,9.72,2217.75,2217.75,0.0
35036,2022-12-31 23:00:00,2177.68,40.07,103.02,2187.05,-9.37,2217.75,2217.75,0.0
35037,2022-12-31 23:15:00,2180.85,36.90,93.12,2177.68,3.17,2217.75,2217.75,0.0


# 代价函数3

In [7]:
# 假设n是维度大小
# n = 100
n = len(data_anly)
M = 30
power = 0.3*2254.4
cap = 0.3*2254.4

P_W_list = 	list(data_anly['Measured & Upscaled'][:n]) # 对原数据进行截取
result_file_route = 'ch5_data/analyse_all_dates/range_width_30.csv'

# P_W_list = [1000,1000,4000,1000,2500,5000,6000,4500,-50,10000,13000,500]
p_bss_values,E_values = solver(P_W_list,n,M,'range',power,cap) 

# P_WC_values = list(np.array(p_bss_values) + np.array(P_W_list[:n-M]))
datetime_list = list(data_anly['Datetime'][:n-M]) 
P_W_list_original = list(data_anly['Measured & Upscaled'][:n-M])
df_test = generate_analyse_dataframe(datetime_list,P_W_list_original,p_bss_values,E_values)
df_test.to_csv(result_file_route)
df_test

Optimizing: 100%|██████████| 35010/35010 [23:29<00:00, 24.83it/s]   


原波动是1431452.8
优化后的波动是936206.0


,Datetime,P_W_list,p_bss_values,E,P_W_shift,original_flucuation,P_CW,P_CW_shift,after_flucuation
0,2022-01-01 00:00:00,1803.48,0.00,338.16,1803.48,0.00,1803.48,1803.48,0.00
1,2022-01-01 00:15:00,1962.39,-208.91,386.84,1803.48,158.91,1753.48,1803.48,-50.00
2,2022-01-01 00:30:00,1801.63,-98.15,409.71,1962.39,-160.76,1703.48,1753.48,-50.00
3,2022-01-01 00:45:00,1824.41,-170.93,449.54,1801.63,22.78,1653.48,1703.48,-50.00
4,2022-01-01 01:00:00,1661.57,-58.09,463.07,1824.41,-162.84,1603.48,1653.48,-50.00
...,...,...,...,...,...,...,...,...,...
35005,2022-12-31 15:15:00,2216.53,0.00,140.44,2217.75,-1.22,2216.53,2217.75,-1.22
35006,2022-12-31 15:30:00,2213.83,0.00,140.44,2216.53,-2.70,2213.83,2216.53,-2.70
35007,2022-12-31 15:45:00,2210.98,0.00,140.44,2213.83,-2.85,2210.98,2213.83,-2.85
35008,2022-12-31 16:00:00,2162.67,0.00,140.44,2210.98,-48.31,2162.67,2210.98,-48.31


# 代价函数4

In [6]:
# 假设n是维度大小
# n = 100
n = len(data_anly)
M = 30
power = 0.3*2254.4
cap = 0.3*2254.4

P_W_list = 	list(data_anly['Measured & Upscaled'][:n]) # 对原数据进行截取
result_file_route = 'ch5_data/analyse_all_dates/range_with_halfE.csv'

# P_W_list = [1000,1000,4000,1000,2500,5000,6000,4500,-50,10000,13000,500]
p_bss_values,E_values = solver(P_W_list,n,M,'range_with_E0.5',power,cap) 

# P_WC_values = list(np.array(p_bss_values) + np.array(P_W_list[:n-M]))
datetime_list = list(data_anly['Datetime'][:n-M]) 
P_W_list_original = list(data_anly['Measured & Upscaled'][:n-M])
df_test = generate_analyse_dataframe(datetime_list,P_W_list_original,p_bss_values,E_values)
df_test.to_csv(result_file_route)
df_test

Optimizing: 100%|██████████| 35010/35010 [1:18:51<00:00,  7.40it/s] 


原波动是1431452.8
优化后的波动是1038594.9199999999


,Datetime,P_W_list,p_bss_values,E,P_W_shift,original_flucuation,P_CW,P_CW_shift,after_flucuation
0,2022-01-01 00:00:00,1803.48,0.00,338.16,1803.48,0.00,1803.48,1803.48,0.00
1,2022-01-01 00:15:00,1962.39,-186.96,381.72,1803.48,158.91,1775.43,1803.48,-28.05
2,2022-01-01 00:30:00,1801.63,-76.20,399.48,1962.39,-160.76,1725.43,1775.43,-50.00
3,2022-01-01 00:45:00,1824.41,-148.98,434.19,1801.63,22.78,1675.43,1725.43,-50.00
4,2022-01-01 01:00:00,1661.57,-36.14,442.61,1824.41,-162.84,1625.43,1675.43,-50.00
...,...,...,...,...,...,...,...,...,...
35005,2022-12-31 15:15:00,2216.53,0.00,338.16,2217.75,-1.22,2216.53,2217.75,-1.22
35006,2022-12-31 15:30:00,2213.83,0.00,338.16,2216.53,-2.70,2213.83,2216.53,-2.70
35007,2022-12-31 15:45:00,2210.98,0.00,338.16,2213.83,-2.85,2210.98,2213.83,-2.85
35008,2022-12-31 16:00:00,2162.67,0.00,338.16,2210.98,-48.31,2162.67,2210.98,-48.31
